In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.parse_csv import Parser
import matplotlib.pyplot as plt
import seaborn as sns
from metrics.MQAGEval import MQAGEval
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

In [3]:
sns.set_style("darkgrid")
sns.set_palette('husl')

In [4]:
parser = Parser()
evaluator = MQAGEval()

Initializing MQAG Evaluator...
MQAG (race) initialized to cpu
MQAGScoreEval initialized to cpu


In [5]:
def get_scores(dir):
    mqag = [[], []]

    for i in tqdm(range(20), desc='getting bidirs'):
        f_path = f'{dir}/run{i+1}.csv'
        try:
            m1, m2 = parser.parse_free(f_path)
            mqag[0].append(evaluator.aggregate(m1, verbose=True))
            mqag[1].append(evaluator.aggregate(m2, verbose=True))
        except Exception:
            print(Exception)
            print(f'error at: {f_path}')
            mqag[0].append(np.nan)
            mqag[1].append(np.nan)

    return mqag

In [6]:
paths = ['/Users/aryanshrivastava/Desktop/LLMWargamingConfidence/logging/outputs/v4/gpt3.5turbo-free-False-20-1.0',
         '/Users/aryanshrivastava/Desktop/LLMWargamingConfidence/logging/outputs/v4/gpt3.5turbo-free-True-20-1.0',
         '/Users/aryanshrivastava/Desktop/LLMWargamingConfidence/logging/outputs/v4/gpt4-free-False-20-1.0',
         '/Users/aryanshrivastava/Desktop/LLMWargamingConfidence/logging/outputs/v4/gpt4-free-True-20-1.0']

In [ ]:
dfs = []
for path in paths:
    data = get_scores(path)
    df = pd.DataFrame(data).T
    df.columns = ['Move 1', 'Move 2']

    dfs.append(df)

for i, path in enumerate(paths):
    path = re.sub('outputs', 'consistency', path) + '/bidirconsistency.csv'
    dfs[i].to_csv(path)